In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (c:\ProgramData\anaconda3\envs\tmu\lib\site-packages\google\protobuf\internal\__init__.py)

In [ ]:
tf.config.list_physical_devices("GPU")